# Calcul du nombre de prélèvements CVM non conforme par commune et par année

L'objectif de ce notebook est de partir de la liste de communes cog_communes, et pour chaque commune et chaque année, calculer le nombre de prélèvements non conformes pour le CVM.

Il y aura plusieurs aggrégations à faire :

- commune (inseecommune) peut avoir plusieurs UDIs (cdreseau) **ET** un UDI peut avoir plusieurs communes (inseecommune) 
- un prélèvement (referenceprel) peut être rattaché à plusieurs UDIs (cdreseau)
- un prélèvement (referenceprel) peut être composé de plusieurs paramètres (cdparametresiseeaux) ; mais dans le cas du CVM, il y a un seul paramètre selon la catégorisation de Pauline, donc c'est plus simple




In [ ]:
%load_ext sql
%sql duckdb:///../../database/data.duckdb
%config SqlMagic.displaylimit = 10

In [ ]:
%%sql
SELECT * FROM int__lien_commune_cdreseau
LIMIT 5 

In [ ]:
%%sql
SELECT * FROM int__lien_cdreseau_refreneceprel
LIMIT 5 

In [ ]:
%%sql
SELECT * FROM int__resultats_udi_communes
LIMIT 5

## Faire la liste complètes des communes (pour chaque catégorie et années)

%%sql --save list_communes_uid
WITH
annees AS (
    SELECT unnest(generate_series(2020, 2024)) as annee
    ),

cat AS (
    SELECT categorie FROM int__mapping_category_simple GROUP BY 1 
    ),

year_cat AS (   
    SELECT
     annee, categorie
    FROM 
    annees
    CROSS JOIN
    cat 
)
    
SELECT DISTINCT
    annee, 
    categorie,
    inseecommune ,
    cdreseau,
 FROM
      int__resultats_udi_communes
 FULL OUTER JOIN
       year_cat
 ON
    udi.year = year_cat.annee

In [ ]:
%%sql --with list_communes_uid
SELECT
    *
FROM
   list_communes_uid
WHERE
    inseecommune = '07194'
    AND categorie = 'cvm'

## Test cog pour le final

In [ ]:
%%sql 
WITH
cog AS (
    SELECT 
      DEP AS code_departement,
      REG AS code_region,
      COM AS inseecommune,
    FROM 
      cog_communes
    WHERE
        TYPECOM = 'COM'
)
    
SELECT
        commune_code_insee,
        COUNT(code_departement) AS nb_code_departement,
        COUNT(code_region) AS nb_code_region,
FROM
    cog
GROUP BY 
       commune_code_insee
HAVING 
    nb_code_departement > 1 OR nb_code_region >1

## Les résultats par communes - Bilan annuel

In [ ]:
%%sql 
 SELECT 
   *
 FROM
   int__resultats_udi_communes
WHERE
    inseecommune = '07194'
    AND categorie = 'cvm'

In [ ]:
%%sql --save int__resultats_communes

WITH communes_year AS (
    SELECT 
     de_partition as annee,
     categorie,
     inseecommune,
     SUM(1) AS nb_analyses,
     SUM(case
            when limitequal_float is not NULL and valtraduite >= limitequal_float then 1
            else 0
        end ) AS nb_analyses_not_ok,
     SUM(case
            when limitequal_float is not NULL and valtraduite < limitequal_float then 1
            else 0
        end ) AS nb_analyses_ok,    
    FROM
      int__resultats_udi_communes
    GROUP BY 
        1,2,3
)

SELECT
    annee,
    inseecommune,
    coalesce(nb_analyses,0) AS nb_analyses,
    coalesce(nb_analyses_not_ok,0) AS nb_analyses_not_ok ,
    coalesce(nb_analyses_ok,0) AS nb_analyses_ok,
    CASE WHEN categorie="cvm" AND coalesce(nb_analyses,0) = 0 THEN 'Pas recherché'
         WHEN categorie="cvm" AND coalesce(nb_analyses,0) > 0  AND coalesce(nb_analyses_ok,0) = 0 THEN 'jamais quantifié'
         WHEN categorie="cvm" AND coalesce(nb_analyses,0) > 0  AND coalesce(nb_analyses_not_ok,0) > 1 THEN '> 0,5 µg/L'
         WHEN categorie="cvm" AND coalesce(nb_analyses,0) > 0  AND coalesce(nb_analyses_ok,0) > 0 THEN '<= 0,5 µg/L'
         ELSE 'Not CVM - other'
    END AS resultat
FROM  
    communes_year
ORDER BY 
    annee

###  Join avec list_communes_uid pour avoir la liste complète

In [ ]:
### SQL FINAL int__resultats_ALL_communes

In [ ]:
%%sql --save int__resultats_ALL_communes --with  list_communes_uid --with int__resultats_communes
SELECT
    annees.annee,
    commune_code_insee,
    coalesce(nb_analyses,0) AS nb_analyses,
    coalesce(nb_analyses_not_ok,0) AS nb_analyses_not_ok ,
    coalesce(nb_analyses_ok,0) AS nb_analyses_ok,
    CASE WHEN coalesce(nb_analyses,0) = 0 THEN 'Pas recherché'
         WHEN coalesce(nb_analyses,0) > 0  AND coalesce(nb_analyses_ok,0) = 0 THEN 'jamais quantifié'
         WHEN coalesce(nb_analyses,0) > 0  AND coalesce(nb_analyses_not_ok,0) > 1 THEN '> 0,5 µg/L'
         WHEN coalesce(nb_analyses,0) > 0  AND coalesce(nb_analyses_ok,0) > 0 THEN '<= 0,5 µg/L'
         ELSE 'check case when'
    END AS resultat
FROM  
 list_communes_uid
LEFT JOIN 
 int__resultats_udi_communes
ON
    list_communes_uid.annee =  int__resultats_udi_communes.annee
    AND list_communes_uid.categorie =  int__resultats_udi_communes.categorie
    AND  list_communes_uid.inseecommune  =  int__resultats_udi_communes.inseecommune

In [ ]:
%%sql --save int__resultats_ALL_communes --with int__resultats_communes
WITH 
/* list_communes_uid  */
annees AS (
    SELECT unnest(generate_series(2020, 2024)) as annee
    ),

cat AS (
    SELECT categorie FROM int__mapping_category_simple GROUP BY 1 
    ),

year_cat AS (   
    SELECT
     annee, categorie
    FROM 
    annees
    CROSS JOIN
    cat 
),

list_communes_uid AS (    
SELECT DISTINCT
    annee, 
    categorie,
    inseecommune ,
    cdreseau,
 FROM
      int__resultats_udi_communes
 FULL OUTER JOIN
       year_cat
 ON
    udi.year = year_cat.annee ),
/* END  list_communes_uid  */


    
SELECT
    list_communes_uid.annee,
    list_communes_uid.inseecommune,
    list_communes_uid.categorie,
    coalesce(nb_analyses,0) AS nb_analyses,
    coalesce(nb_analyses_not_ok,0) AS nb_analyses_not_ok ,
    coalesce(nb_analyses_ok,0) AS nb_analyses_ok,
    CASE WHEN coalesce(nb_analyses,0) = 0 THEN 'Pas recherché'
         WHEN coalesce(nb_analyses,0) > 0  AND coalesce(nb_analyses_ok,0) = 0 THEN 'jamais quantifié'
         WHEN coalesce(nb_analyses,0) > 0  AND coalesce(nb_analyses_not_ok,0) > 1 THEN '> 0,5 µg/L'
         WHEN coalesce(nb_analyses,0) > 0  AND coalesce(nb_analyses_ok,0) > 0 THEN '<= 0,5 µg/L'
         ELSE 'check case when'
    END AS resultat
FROM  
 list_communes_uid
LEFT JOIN 
 int__resultats_communes
ON
    list_communes_uid.annee =  int__resultats_communes.annee
    AND list_communes_uid.categorie =  int__resultats_communes.categorie
    AND  list_communes_uid.inseecommune  =  int__resultats_communes.inseecommune


In [ ]:
%%sql --with int__resultats_ALL_communes
SELECT 
    *
FROM
    int__resultats_ALL_communes
WHERE
 LIST_REF_UDI_YEAR.categorie = 'cvm'

###  Join avec cog pour avoir le nom des communes

In [ ]:
%%sql --save int__resultats_ALL_communes_name --with int__resultats_ALL_communes
WITH 
cog AS (
    SELECT
      DEP AS code_departement,
      REG AS code_region,
      COM AS inseecommune,
    FROM 
      cog_communes
    WHERE
     TYPECOM= 'COM'
),

    SELECT
      udi.*,
      cog.code_departement,
      cog.code_region
    FROM
      int__resultats_ALL_communes
    LEFT JOIN 
      cog
    ON 
      int__resultats_ALL_communes.inseecommune = cog.inseecommune

In [ ]:
%%sql --with int__resultats_ALL_communes_name
SELECT 
    *
FROM
    int__resultats_ALL_communes_name
WHERE
     categorie = 'cvm'

# Check

In [ ]:
%%sql
SELECT
    de_partition as year,
    COUNT(DISTINCT inseecommune) AS nb_inseecommune
FROM 
 edc_communes
GROUP BY 
    1 
ORDER BY 
   1

In [ ]:
%%sql --with int__resultats_ALL_communes_name  
 SELECT 
    annee,
    COUNT(DISTINCT inseecommune) AS nb_inseecommune
 FROM
   int__resultats_ALL_communes_name
GROUP BY 
    1

In [ ]:
#### commune_code_insee = '07194'

In [ ]:
%%sql --with int__resultats_udi_communes 
 SELECT 
   *
 FROM
   int__resultats_udi_communes
WHERE
    inseecommune = '07194'
    AND categorie = 'cvm'
ORDER BY 
    annee

In [ ]:
%%sql --with int__resultats_communes 
 SELECT 
   *
 FROM
   int__resultats_communes
WHERE
    inseecommune = '07194'
    AND categorie = 'cvm'
ORDER BY 
    annee

In [ ]:
%%sql --with int__resultats_ALL_communes_name 
 SELECT 
   *
 FROM
   int__resultats_ALL_communes_name
WHERE
    inseecommune = '07194'
    AND categorie = 'cvm'
ORDER BY 
    annee

In [ ]:
#### commune_code_insee = 'XX'

In [ ]:
%%sql --with int__resultats_udi_communes 
 SELECT 
   *
 FROM
   int__resultats_udi_communes
WHERE
    inseecommune = 'X'
    AND categorie = 'cvm'
ORDER BY 
    annee

In [ ]:
%%sql --with int__resultats_ALL_communes_name 
 SELECT 
   *
 FROM
   int__resultats_ALL_communes_name
WHERE
    inseecommune = 'X'
    AND categorie = 'cvm'
ORDER BY 
    annee